In [1]:
import numpy as np
import pandas as pd
import math

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
pd.options.display.float_format = '{:2f}'.format
# pd.options.display.max_rows = None  # 모든 row 내용 print
pd.options.display.max_rows = 20

### Load the dataset with pandas
The car data set we will be using in this lab is provided as a comma separated file without a header row.  In order for each column to have a meaningful header name we must provide it.  We get the information about the columns from the [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/automobile).

We will use the features of the car, to try to predict its price.

In [3]:
# Provide the names for the columns since the CSV file with the data does not have a header row.
feature_names = ['symboling', 'normalized-losses', 'make', 'fuel-type',
        'aspiration', 'num-doors', 'body-style', 'drive-wheels',
        'engine-location', 'wheel-base', 'length', 'width', 'height', 'weight',
        'engine-type', 'num-cylinders', 'engine-size', 'fuel-system', 'bore',
        'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
        'highway-mpg', 'price']

In [4]:
# Load in the data from a CSV file that is comma separated.
car_data = pd.read_csv('https://storage.googleapis.com/mledu-datasets/cars_data.csv',
                     sep=',', names=feature_names, header=None, encoding='latin-1')

 We'll then randomize the data, just to be sure not to get any pathological ordering effects that might harm the performance of Stochastic Gradient Descent.

In [5]:
car_data = car_data.reindex(np.random.permutation(car_data.index))
# or equivalently,
# car_data = car_data.sample(frac=1)

In [6]:
print("Data set loaded. Num examples: ", len(car_data))

Data set loaded. Num examples:  205


In [7]:
car_data.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
182,2,122,volkswagen,diesel,std,two,sedan,fwd,front,97.300000,...,97,idi,3.01,3.40,23.000000,52,4800,37,46,7775
55,3,150,mazda,gas,std,two,hatchback,rwd,front,95.300000,...,70,4bbl,?,?,9.400000,101,6000,17,23,10945
145,0,102,subaru,gas,turbo,four,sedan,4wd,front,97.000000,...,108,mpfi,3.62,2.64,7.700000,111,4800,24,29,11259
45,0,?,isuzu,gas,std,four,sedan,fwd,front,94.500000,...,90,2bbl,3.03,3.11,9.600000,70,5400,38,43,?
160,0,91,toyota,gas,std,four,sedan,fwd,front,95.700000,...,98,2bbl,3.19,3.03,9.000000,70,4800,38,47,7738


This is a really small dataset! Only 205 examples.

For simplicity in this codelab, we do not split the data further into training and validation. But you MUST do this on real datasets, or else you will overfit to your single dataset.

## Task 0: Use pandas to explore and prepare the data

- Use Pandas to inspect the data and manually curate a list of numeric_feature_names and categorical_feature_names.


Useful functions:
- `type()` called on any Python object describes the type of the object
- `dataframe[4:7]` pulls out rows 4, 5, 6 in a Pandas dataframe
- `dataframe[['mycol1', 'mycol2']]` pulls out the two requested columns into a new Pandas dataframe
- `dataframe['mycol1']` returns a Pandas series -- not a dataframe!
- `dataframe.describe()` prints out statistics for each dataframe column

In [8]:
car_data[4:7]

,symboling,normalized-losses,make,fuel-type,aspiration,num-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
160,0,91,toyota,gas,std,four,sedan,fwd,front,95.700000,...,98,2bbl,3.19,3.03,9.000000,70,4800,38,47,7738
23,1,118,dodge,gas,turbo,two,hatchback,fwd,front,93.700000,...,98,mpfi,3.03,3.39,7.600000,102,5500,24,30,7957
142,0,102,subaru,gas,std,four,sedan,fwd,front,97.200000,...,108,2bbl,3.62,2.64,9.500000,82,4400,28,33,7775


In [9]:
car_data.describe()

,symboling,wheel-base,length,width,height,weight,engine-size,compression-ratio,city-mpg,highway-mpg
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,10.142537,25.219512,30.751220
std,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,3.972040,6.542142,6.886443
min,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,7.000000,13.000000,16.000000
25%,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,8.600000,19.000000,25.000000
50%,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,9.000000,24.000000,30.000000
75%,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,9.400000,30.000000,34.000000
max,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,23.000000,49.000000,54.000000


In [10]:
LABEL = 'price'

numeric_feature_names = ['symboling', 'normalized-losses', 'wheel-base',
        'length', 'width', 'height', 'weight', 'engine-size', 'horsepower',
        'peak-rpm', 'city-mpg', 'highway-mpg', 'bore', 'stroke',
         'compression-ratio']

categorical_feature_names = list(set(feature_names) - set(numeric_feature_names) - set([LABEL]))

categorical_feature_names

assert len(numeric_feature_names) == 15
assert len(categorical_feature_names) == 10

In [11]:
# The correct solution will pass these assert statements.
assert len(numeric_feature_names) == 15
assert len(categorical_feature_names) == 10

In [12]:
# Run to inspect numeric features.
car_data[numeric_feature_names]

,symboling,normalized-losses,wheel-base,length,width,height,weight,engine-size,horsepower,peak-rpm,city-mpg,highway-mpg,bore,stroke,compression-ratio
182,2,122,97.300000,171.700000,65.500000,55.700000,2261,97,52,4800,37,46,3.01,3.40,23.000000
55,3,150,95.300000,169.000000,65.700000,49.600000,2380,70,101,6000,17,23,?,?,9.400000
145,0,102,97.000000,172.000000,65.400000,54.300000,2510,108,111,4800,24,29,3.62,2.64,7.700000
45,0,?,94.500000,155.900000,63.600000,52.000000,1909,90,70,5400,38,43,3.03,3.11,9.600000
160,0,91,95.700000,166.300000,64.400000,53.000000,2094,98,70,4800,38,47,3.19,3.03,9.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,2,121,88.400000,141.100000,60.300000,53.200000,1488,61,48,5100,47,53,2.91,3.03,9.500000
180,-1,90,104.500000,187.800000,66.500000,54.100000,3131,171,156,5200,20,24,3.27,3.35,9.200000
178,3,197,102.900000,183.500000,67.700000,52.000000,2976,171,161,5200,20,24,3.27,3.35,9.300000
174,-1,65,102.400000,175.600000,66.500000,54.900000,2480,110,73,4500,30,33,3.27,3.35,22.500000


In [13]:
car_data[numeric_feature_names].describe()

,symboling,wheel-base,length,width,height,weight,engine-size,city-mpg,highway-mpg,compression-ratio
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,25.219512,30.751220,10.142537
std,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,6.542142,6.886443,3.972040
min,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,13.000000,16.000000,7.000000
25%,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,19.000000,25.000000,8.600000
50%,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,24.000000,30.000000,9.000000
75%,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,30.000000,34.000000,9.400000
max,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,49.000000,54.000000,23.000000


In [15]:
# Coerce the numeric features to numbers. This is necessary because the model crashes because not all the values are numeric.
for feature_name in numeric_feature_names + [LABEL]:
    car_data[feature_name] = pd.to_numeric(car_data[feature_name], errors ='coerce')

"""
errors: error는 총 3개의 옵션이 존재합니다.
- errors = 'ignore' -> 만약 숫자로 변경할 수 없는 데이터라면 숫자로 변경하지 않고 원본 데이터를 그대로 반환합니다.
- errors = 'coerce' -> 만약 숫자로 변경할 수 없는 데이터라면 기존 데이터를 지우고 NaN으로 설정하여 반환합니다.
- errors = 'raise' -> 만약 숫자로 변경할 수 없는 데이터라면 에러를 일으키며 코드를 중단합니다."""
    
# Fill missing values with 0.
# Is this an OK thing to do? You may want to come back and revisit this decision later.
car_data.fillna(0, inplace=True)

## Task 1: Make your best model with numeric features. No normalization allowed.

Modify the model provided below to achieve the lowest eval loss. You may want to change various hyperparameters:
- learning rate
- choice of optimizer
- hidden layer dimensions -- make sure your choice here makes sense given the number of training examples
- batch size
- num training steps
- (anything else you can think of changing)

Do not use the `normalizer_fn` arg on `numeric_column`.